In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import datasets
# from sklearn.datasets import load_boston
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLSResults
from math import sqrt
import random
import sys

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

sns.set(style="darkgrid")

In [5]:
# load data frames
df_train_1 = pd.read_csv('TrainData.csv')
df_test_1 = pd.read_csv('TestData.csv')
print(df_train_1.head())
print(df_test_1.head())

   0      1      2  3   4  5       6  7  8   9  10  11   12      13     target
0  1  34.75  15.00  2  12  7   5.375  A  1   9   1   2    0   135.0      Churn
1  1  19.17   4.00  1   3  4   1.000  B  0   0   1   2  360  1001.0  Not churn
2  1  20.25   9.96  2  10  2   0.000  A  0   0   0   2    0     1.0      Churn
3  1    NaN  28.00  1   8  4  28.500  A  1  40   0   2    0    16.0      Churn
4  1  22.58   1.50  1   6  4   0.540  B  0   0   1   2  120    68.0  Not churn
   0      1       2  3   4  5      6  7  8  9  10  11   12   13
0  1  32.08   4.000  1  13  4  1.500  B  0  0   1   2  120    1
1  0  23.50   1.500  2   9  4  0.875  B  0  0   1   2  160    1
2  0  33.25   2.500  1   8  4  2.500  B  0  0   1   2    0    3
3  1  74.83  19.000  1   1  1  0.040  B  1  2   0   2    0  352
4  1  20.17   5.625  2   9  4  1.710  A  0  0   0   1  120    1


Задание 1 (0.15 балла). Проверьте, есть ли в тренировочных и в тестовых данных пропуски? Укажите количество столбцов тренировочной выборки, имеющих пропуски

In [75]:
train_missing = df_train_1.isnull().sum() # get missing in train
test_missing = df_test_1.isnull().sum() # get missing in test

num_train_columns_with_missing = (train_missing > 0).sum() # count num of columns with missing data
print(num_train_columns_with_missing)

2


In [8]:
df_train_2 = pd.read_csv('train.csv')
df_test_2 = pd.read_csv('test.csv')
print(df_train_2.head())
print(df_test_2.head())

   battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0           1057     1          2.7         0   3       1          41    0.1   
1           1619     1          1.9         0   2       1          21    0.4   
2           1028     1          2.8         0   1       1          30    0.1   
3           1994     1          0.8         1   2       1           7    0.6   
4           1603     1          0.5         1   1       1          17    0.5   

   mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  talk_time  \
0         97        4  ...        397       891  2033    16     9          2   
1        138        1  ...         46       562  1641     7     2         13   
2        193        1  ...       1801      1923  1424    18     4         13   
3         88        8  ...        667       711   454    11     8          6   
4        114        2  ...        430      1207  1637    17     3         11   

   three_g  touch_screen wifi  price_r

Задание 1 (0.25 балла).

В скольких столбцах таблицы среднее значение измеряется в сотнях? (не в десятках и не в тысячах).

In [77]:
# Отфильтровывание только числовых столбцов в выборке
df_train_numeric = df_train_2.select_dtypes(include=['number'])
df_test_numeric = df_test_2.select_dtypes(include=['number'])

# Вычисление среднего значения для каждого числового столбца в выборке
train_mean_values = df_train_numeric.mean()
test_mean_values = df_test_numeric.mean()

columns_in_hundreds = train_mean_values[(train_mean_values >= 100) & (train_mean_values < 1000)].count()
print(columns_in_hundreds)

2


In [58]:
df_train_3 = pd.read_csv('Data_train.csv')
df_test_3 = pd.read_csv('Data_test.csv')
print(df_train_3.head())

       type    group          education      meal preparation course  score-1  \
0      wild  group B   some high school  standard          completed       63   
1  domestic  group C  bachelor's degree  standard               none       67   
2  domestic  group C       some college  standard          completed       69   
3  domestic  group B   some high school  standard               none       62   
4      wild  group E       some college   reduced               none       93   

   score-2  score-3  
0       67       67  
1       69       75  
2       90       88  
3       64       66  
4       90       83  


In [47]:
print(df_test_3.head())

       type    group           education      meal preparation course
0      wild  group D        some college  standard               none
1      wild  group C     master's degree   reduced          completed
2      wild  group B        some college   reduced          completed
3      wild  group A  associate's degree   reduced               none
4  domestic  group B        some college  standard          completed


In [50]:
print(df_train_3.shape)
print(df_train_3.describe())
print(df_train_3.describe(exclude=['number']))

(700, 8)
          score-1     score-2     score-3
count  700.000000  700.000000  700.000000
mean    66.028571   69.038571   67.828571
std     15.140138   14.589664   15.152323
min      0.000000   17.000000   10.000000
25%     56.000000   59.000000   57.000000
50%     66.000000   70.000000   68.000000
75%     77.000000   79.000000   78.000000
max    100.000000  100.000000  100.000000
            type    group           education      meal preparation course
count        700      680                 700       700                700
unique         2        5                   6         2                  2
top     domestic  group C  associate's degree  standard               none
freq         357      207                 154       444                442


In [57]:
print(df_test_3.shape)
print(df_test_3.describe())

(300, 5)
            type    group     education      meal preparation course
count        300      288           300       300                300
unique         2        5             6         2                  2
top     domestic  group C  some college  standard               none
freq         161      103            76       201                200


Задание 1 (0.25 балла). Заполните пропуски в столбце уникальной категорией, если столбец с пропуском категориальный, и средним значением, если столбец числовой. Заполняйте одновременно и df_train, и df_test - одинаковым образом. В ответе укажите количество различных значений, потребовавшихся для заполнения пропусков (это равно количеству новых уникальных категорий плюс количество средних значений для заполнения пропусков в числовых столбцах).



In [60]:
train_missing_3 = df_train_3.isnull().sum() # get missing in train
test_missing = df_test_3.isnull().sum() # get missing in test
num_train_columns_with_missing = (train_missing_3 > 0).sum()
num_test_columns_with_missing = (test_missing > 0).sum()
print(num_train_columns_with_missing)
print(num_test_columns_with_missing)

numeric_columns_train = df_train_3.select_dtypes(include=['number']).columns
numeric_columns_test = df_test_3.select_dtypes(include=['number']).columns
categorical_columns_train = df_train_3.select_dtypes(exclude=['number']).columns
categorical_columns_test = df_test_3.select_dtypes(exclude=['number']).columns

mean_values_train = df_train_3[numeric_columns_train].mean()
mean_values_test = df_test_3[numeric_columns_test].mean()
df_train_3[numeric_columns_train] = df_train_3[numeric_columns_train].fillna(mean_values_train)
df_test_3[numeric_columns_test] = df_test_3[numeric_columns_test].fillna(mean_values_test)

unique_category = 'unknown'

df_train_3[categorical_columns_train] = df_train_3[categorical_columns_train].fillna(unique_category)
df_test_3[categorical_columns_test] = df_test_3[categorical_columns_test].fillna(unique_category)


0
0


Задание 2 (0.3 балла)
Если у телефона нет ни опции 3G, ни опции 4G, ни Wifi - то у пользователей нет доступа к интернету. Создайте колонку is_internet и поставьте туда 0, если у телефона нет ни одной из трех перечисленных опций, и 1 иначе.

Такую же колонку создайте в тестовых данных.

Какая доля телефонов (из train.csv) не имеет доступа к интернету? Ответ округлите до сотых.

In [61]:
df_train_2['is_internet'] = df_train_2.apply(lambda row: 1 if row['three_g'] == 1 or row['four_g'] == 1 or row['wifi'] == 1 else 0, axis=1)
df_test_2['is_internet'] = df_test_2.apply(lambda row: 1 if row['three_g'] == 1 or row['four_g'] == 1 or row['wifi'] == 1 else 0, axis=1)

# Вычисление доли телефонов без доступа к интернету
no_internet_share = (df_train_2['is_internet'] == 0).mean()

# Округление до сотых
no_internet_share_rounded = round(no_internet_share, 2)

# Вывод результата
print("Доля телефонов без доступа к интернету:", no_internet_share_rounded)

Доля телефонов без доступа к интернету: 0.12


a) (0.05 балла). В столбце с наибольшим количеством пропусков заполните пропуски средним значением по столбцу. В ответ запишите значение вычисленного среднего.  Ответ округлите до десятых.


In [78]:
column_with_most_missing = train_missing.idxmax()
mean_value = df_train_1[column_with_most_missing].mean()
df_train_1[column_with_most_missing].fillna(mean_value, inplace=True)
updated_mean_value = df_train_1[column_with_most_missing].mean()

# Округление до десятых
updated_mean_value_rounded = round(updated_mean_value, 1)

# Вывод результата
print(f"Столбец с наибольшим количеством пропусков: {column_with_most_missing}")
print(f"Заполненное среднее значение: {updated_mean_value_rounded}")

Столбец с наибольшим количеством пропусков: 1
Заполненное среднее значение: 31.3


b) (0.1 балла). Найдите строки в тренировочных данных, где пропуски стоят в столбце с наименьшим количеством пропусков. Удалите эти строки. Сколько строк вы удалили?



In [82]:
# Подсчет пропусков в каждой строке
missing_counts_per_row = df_train_1.isnull().sum(axis=1)
# Находим столбец с наименьшим количеством пропусков
column_with_least_missing = missing_counts_per_row.idxmin()
# Выбираем строки с пропусками в этом столбце
rows_to_drop = df_train_1[missing_counts_per_row > 0]
# Удаляем выбранные строки
df_train_cleaned = df_train_1.drop(rows_to_drop.index)
# Подсчитываем количество удаленных строк
num_rows_dropped = len(rows_to_drop)
print(num_rows_dropped)

3


In [84]:
print(df_train_1.describe(exclude=['number']))
unique_values = df_train_1.apply(lambda x: x.unique())
print(unique_values)

          7     target
count   513        513
unique    2          4
top       A  Not churn
freq    279        278
0                                                    [1, 0]
1         [34.75, 19.17, 20.25, 31.28322514908239, 22.58...
2         [15.0, 4.0, 9.96, 28.0, 1.5, 1.0, 3.165, 7.585...
3                                                 [2, 1, 3]
4           [12, 3, 10, 8, 6, 11, 14, 9, 13, 1, 4, 7, 2, 5]
5                                  [7, 4, 2, 5, 8, 1, 9, 3]
6         [5.375, 1.0, 0.0, 28.5, 0.54, 5.0, 3.165, 7.58...
7                                                    [A, B]
8                                                    [1, 0]
9         [9, 0, 40, 2, 3, 15, 1, 7, 4, 5, 6, 11, 14, 20...
10                                                   [1, 0]
11                                                [2, 1, 3]
12        [0, 360, 120, 167, 290, 380, 60, 340, 220, 100...
13        [135.0, 1001.0, 1.0, 16.0, 68.0, 501.0, 2280.0...
target                 [Churn, Not churn, Cho

Переведите столбец с целевой переменной в бинарные значения по правилу: Churn - 1, Not churn - 0. Исправьте опечатки в названиях категорий целевой переменной (до того, как переводить их в бинарные значения).

Сколько опечаток вы исправили?

In [85]:
need_to_fix = df_train_1['target'].isin(['Chorn', 'Not chorn']).sum()
print(need_to_fix)

df_train_1['target'] = df_train_1['target'].replace({'Chorn': 'Churn', 'Not chorn': 'Not churn'})

# Преобразуем значения в бинарные
df_train_1['target'] = df_train_1['target'].map({'Churn': 1, 'Not churn': 0})

2


Верно ли, что если значение единственного категориального признака в таблице равно A, то клиент уйдет из компании? (target = 1) В ответ запишите “да” или “нет” без кавычек.

In [89]:
# Фильтруем DataFrame по условиям
filtered_rows = df_train_1[(df_train_1['7'] == 'A') & (df_train_1['target'] == 0)]

# Подсчитываем количество строк в результате фильтрации
num_rows = len(filtered_rows)
print(num_rows)

61


Вычислите долю ушедших из компании клиентов, для которых значение признака 2 больше среднего значения по столбцу, а значение признака 13 меньше медианы по столбцу. Ответ округлите до сотых.

In [97]:
mean_feature_2 = df_train_1['2'].mean()
median_feature_13 = df_train_1['13'].median()

# Определяем строки, удовлетворяющие условиям
filtered_rows = df_train_1[(df_train_1['2'] > mean_feature_2) & (df_train_1['13'] < median_feature_13)]
rows2 = filtered_rows[filtered_rows['target'] == 1]

# Вычисляем долю ушедших клиентов относительно общего числа строк
churn_ratio = len(rows2) / len(filtered_rows)

# Ответ округляем до сотых
churn_ratio_rounded = round(churn_ratio, 2)

# Выводим результат
print("Доля ушедших клиентов:", churn_ratio_rounded)

Доля ушедших клиентов: 0.51


a) (0.25 балла) Закодируйте категориальные столбцы в тренировочных и тестовых данных при помощи label encoding (категории кодируйте подряд идущими числами, начинающимися с 0).


Сколько столбцов после кодировки стало в тестовых данных?

In [8]:
from sklearn.preprocessing import LabelEncoder

# Загрузка данных
df_train = pd.read_csv('TrainData.csv')
df_test = pd.read_csv('TestData.csv')

# заполнить пропущенные записи и все предыдущие пункты

# Создание объекта LabelEncoder
label_encoder = LabelEncoder()

# Функция для кодировки категориальных столбцов
def encode_categorical(df):
    for column in df.select_dtypes(include=['object']).columns:
        df[column] = label_encoder.fit_transform(df[column])
    return df

# Применение LabelEncoder к тренировочным и тестовым данным
df_train = encode_categorical(df_train)
df_test = encode_categorical(df_test)

# Определение количества столбцов в тестовых данных после кодировки
num_columns_test_encoded = df_test.shape[1]

# Вывод количества столбцов
print("Количество столбцов после кодировки в тестовых данных:", num_columns_test_encoded)

Количество столбцов после кодировки в тестовых данных: 14


b) (0.5 балла) Разбейте тренировочные данные на целевой вектор y, содержащий значения из столбца target, и матрицу объект-признак X, содержащую остальные признаки. Обучите на этих данных логистическую регрессию из sklearn (LogisticRegression) с параметрами по умолчанию. Выведите среднее значение метрики f1-score алгоритма на кросс-валидации с тремя фолдами. Ответ округлите до сотых.

При объявлении модели фиксируйте random_state = 42.

Комментарий: параметры по умолчанию предполагаются следующими

penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer

X = df_train.drop(columns=['target'])
y = df_train['target']

# Создание и обучение модели логистической регрессии
model = LogisticRegression(random_state=42)

# Определение метрики f1-score для кросс-валидации
f1_scorer = make_scorer(f1_score)

# Кросс-валидация с тремя фолдами, добавим обработку ошибок
try:
    scores = cross_val_score(model, X, y, cv=3, scoring=f1_scorer)
    # Вычисление среднего значения метрики f1-score
    mean_f1_score = scores.mean()
    # Округление до сотых
    mean_f1_score_rounded = round(mean_f1_score, 2)
    # Вывод результата
    print("Среднее значение f1-score на кросс-валидации:", mean_f1_score_rounded)
except Exception as e:
    print("Ошибка при выполнении кросс-валидации:", e)

Ошибка при выполнении кросс-валидации: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1196, in fit
    X, y = self._validate_data(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py", line 1106, in check_X_y
    X = check_array(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils

6. a) Подберите значение константы регуляризации C в логистической регрессии, перебирая гиперпараметр от 0.001 до 100 включительно, проходя по степеням 10. Для выбора С примените перебор по сетке по тренировочной выборке (GridSearchCV из библиотеки sklearn.model_selection) с тремя фолдами и метрикой качества - f1-score. Остальные параметры оставьте по умолчанию. В ответ запишите наилучшее среди искомых значение С. (0.25 балла)

При объявлении модели фиксируйте random_state = 42.

Комментарий: параметры по умолчанию предполагаются следующими

penalty='l2', dual=False, tol=0.0001, fit_intercept=True, intercept_scaling=1, class_weight=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None

b) Добавьте в тренировочные и тестовые данные новый признак ‘NEW’, равный произведению признаков ‘7’ и ‘11’.


На тренировочных данных с новым признаком заново с помощью GridSearchCV (с тремя фолдами и метрикой качества - f1-score) подберите оптимальное значение С (перебирайте те же значения С, что и в предыдущих заданиях), в ответ напишите наилучшее качество алгоритма (по метрике f1-score), ответ округлите до сотых. (0.5 балла).

При объявлении модели фиксируйте random_state = 42.

Комментарий: параметры по умолчанию предполагаются следующими

penalty='l2', dual=False, tol=0.0001, fit_intercept=True, intercept_scaling=1, class_weight=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None

c) Теперь вы можете использовать любую модель машинного обучения для решения задачи. Также можете делать любую другую обработку признаков. Ваша задача - получить наилучшее качество по метрике accuracy на тестовых данных. (0.75 балла)


Качество проверяется на представленных тестовых данных.

accuracy >= 0.88 - 0.25 балла
accuracy >= 0.9 - 0.75 балла
Сдайте файл result.csv. Во вложении пример файла для отправки результатов: result.csv
Задание считается выполненным, если загруженный файл прошел предпроверку и получил необходимые баллы. Не обращайте внимание на статус "Не завершено" слева от задания.
Внимание! Учитывается результат последней посылки! Перед завершением теста убедитесь, что вы отправили последним самое точное предсказание.

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

# Загрузка данных
df_train = pd.read_csv('TrainData.csv')
df_test = pd.read_csv('TestData.csv')

# Кодирование категориальных столбцов
label_encoder = LabelEncoder()
for column in df_train.select_dtypes(include=['object']).columns:
    df_train[column] = label_encoder.fit_transform(df_train[column])
    df_test[column] = label_encoder.fit_transform(df_test[column])

# Разделение данных на целевой вектор y и матрицу объект-признак X
X_train = df_train.drop(columns=['target'])
y_train = df_train['target']

# Определение параметров для GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
model = LogisticRegression(random_state=42)

# Создание и обучение GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='f1')
grid_search.fit(X_train, y_train)

# Вывод наилучшего значения C
best_C = grid_search.best_params_['C']
print("Наилучшее значение C:", best_C)

In [ ]:
# Добавление нового признака 'NEW'
df_train['NEW'] = df_train['7'] * df_train['11']
df_test['NEW'] = df_test['7'] * df_test['11']

# Разделение данных на целевой вектор y и матрицу объект-признак X
X_train_new = df_train.drop(columns=['target'])
y_train_new = df_train['target']

# Создание и обучение GridSearchCV
grid_search_new = GridSearchCV(model, param_grid, cv=3, scoring='f1')
grid_search_new.fit(X_train_new, y_train_new)

# Вывод наилучшего значения C
best_C_new = grid_search_new.best_params_['C']
best_f1_score = grid_search_new.best_score_
print("Наилучшее значение C с новым признаком:", best_C_new)
print("Наилучшее значение f1-score с новым признаком:", round(best_f1_score, 2))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Создание и обучение RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_new, y_train_new)

# Предсказание и оценка на тестовых данных
X_test_new = df_test.drop(columns=['target'])
y_test = df_test['target']

y_pred_rf = rf_model.predict(X_test_new)
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print("Точность RandomForestClassifier:", round(rf_accuracy, 2))

# Создание и обучение XGBoost
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train_new, y_train_new)

# Предсказание и оценка на тестовых данных
y_pred_xgb = xgb_model.predict(X_test_new)
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
print("Точность XGBoost:", round(xgb_accuracy, 2))

# Сохранение лучшей модели
best_model = rf_model if rf_accuracy > xgb_accuracy else xgb_model
best_accuracy = max(rf_accuracy, xgb_accuracy)
print("Наилучшая точность модели:", round(best_accuracy, 2))

In [ ]:
# Создание файла с результатами
result_df = pd.DataFrame({'id': df_test.index, 'target': best_model.predict(X_test_new)})
result_df.to_csv('result.csv', index=False)

Разбейте тренировочные данные на целевой вектор y, содержащий значения из столбца price_range, и матрицу объект-признак X, содержащую остальные признаки.

Посмотрите, сколько значений содержит категориальный столбец touch_screen. Значения, встречающиеся меньше, чем в 1% строк, замените на самое частое значение. После этого закодируйте столбец при помощи OneHot-encoding (используйте аргумент drop='first'). Обучайте OHE на тренировочных данных X, но кодируйте при этом и тренировочные, и тестовые данные.  Для кросс-валидации лучше использовать функцию cross_val_score из библиотеки sklearn.model_selection.

Сколько изначально различных значений было в столбце touch_screen?

b (0.4 балла) Обучите на этих данных логистическую регрессию из sklearn (LogisticRegression) с параметрами по умолчанию. Выведите среднее значение метрики f1_score c вариантом усреднения ‘weighted’  (или же 'f1_weighted') алгоритма на кросс-валидации с тремя фолдами. Ответ округлите до сотых.

При объявлении модели задайте random_state = 42.

Комментарий: параметры по умолчанию предполагаются следующими:

penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None

a. Подберите значение константы регуляризации C в логистической регрессии, перебирая гиперпараметр от 0.001 до 100 включительно, проходя по степеням 10. Для выбора С примените перебор по сетке по тренировочной выборке (GridSearchCV из библиотеки sklearn.model_selection) с тремя фолдами и метрикой качества - f1_weighted. Остальные параметры оставьте по умолчанию. В ответ запишите наилучшее среди искомых значение С. (0.25 балла)

При объявлении модели задайте random_state = 42.

Комментарий: параметры по умолчанию предполагаются следующими:

penalty='l2', dual=False, tol=0.0001, fit_intercept=True, intercept_scaling=1, class_weight=None,  solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None

b. Добавьте в тренировочные и тестовые данные новый признак ‘ram_resolution’, вычисляемый по формуле ram * (px_height + px_width)

На тренировочных данных с новым признаком заново с помощью GridSearchCV (с тремя фолдами и метрикой качества - f1_weighted) подберите оптимальное значение С (перебирайте те же значения С, что и в предыдущих заданиях), в ответ напишите наилучшее качество алгоритма по метрике f1_weighted (можно посмотреть, обратившись к полю best_score_ обученного GridSearchCV), ответ округлите до сотых. (0.5 балла).

Комментарий: параметры по умолчанию предполагаются следующими

penalty='l2', dual=False, tol=0.0001, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None

с. Теперь вы можете использовать любую модель машинного обучения для решения задачи. Также можете делать любую другую обработку признаков. Ваша задача - получить наилучшее качество по метрике f1_weighted на тестовых данных. (0.75 балла)

Качество проверяется на представленных тестовых данных.

* f1_weighted >= 0.88 - 0.25 балла

* f1_weighted >= 0.93 - 0.75 балла

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Загрузка данных
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Разделение данных на X и y
X_train = train_data.drop(columns=['price_range'])
y_train = train_data['price_range']

In [ ]:
# Подсчет уникальных значений
touch_screen_counts = X_train['touch_screen'].value_counts()
initial_unique_values = len(touch_screen_counts)

# Замена значений, встречающихся менее 1% строк, на самое частое значение
threshold = 0.01 * len(X_train)
most_frequent_value = touch_screen_counts.idxmax()
X_train['touch_screen'] = X_train['touch_screen'].apply(lambda x: most_frequent_value if touch_screen_counts[x] < threshold else x)

# OneHot-Encoding
encoder = OneHotEncoder(drop='first', sparse=False)
touch_screen_encoded = encoder.fit_transform(X_train[['touch_screen']])
touch_screen_encoded_df = pd.DataFrame(touch_screen_encoded, columns=encoder.get_feature_names_out(['touch_screen']))

# Замена столбца touch_screen на закодированный вариант
X_train = X_train.drop(columns=['touch_screen'])
X_train = pd.concat([X_train, touch_screen_encoded_df], axis=1)

# Применение того же кодирования к тестовым данным
X_test = test_data.drop(columns=['price_range'])
X_test['touch_screen'] = X_test['touch_screen'].apply(lambda x: most_frequent_value if touch_screen_counts.get(x, 0) < threshold else x)
touch_screen_encoded_test = encoder.transform(X_test[['touch_screen']])
touch_screen_encoded_test_df = pd.DataFrame(touch_screen_encoded_test, columns=encoder.get_feature_names_out(['touch_screen']))
X_test = X_test.drop(columns=['touch_screen'])
X_test = pd.concat([X_test, touch_screen_encoded_test_df], axis=1)

print("Изначально различных значений в столбце touch_screen:", initial_unique_values)

In [ ]:
# Обучение модели логистической регрессии
model = LogisticRegression(random_state=42)
f1_scores = cross_val_score(model, X_train, y_train, cv=3, scoring='f1_weighted')
mean_f1_score = f1_scores.mean()
print("Среднее значение f1_score:", round(mean_f1_score, 2))

In [ ]:
from sklearn.model_selection import GridSearchCV

# Определение параметров для GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
model = LogisticRegression(random_state=42)

# Создание и обучение GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='f1_weighted')
grid_search.fit(X_train, y_train)

# Вывод наилучшего значения C
best_C = grid_search.best_params_['C']
print("Наилучшее значение C:", best_C)

In [ ]:
# Добавление нового признака 'ram_resolution'
X_train['ram_resolution'] = X_train['ram'] * (X_train['px_height'] + X_train['px_width'])
X_test['ram_resolution'] = X_test['ram'] * (X_test['px_height'] + X_test['px_width'])

# Создание и обучение GridSearchCV
grid_search_new = GridSearchCV(model, param_grid, cv=3, scoring='f1_weighted')
grid_search_new.fit(X_train, y_train)

# Вывод наилучшего значения C и f1_score
best_C_new = grid_search_new.best_params_['C']
best_f1_score = grid_search_new.best_score_
print("Наилучшее значение C с новым признаком:", best_C_new)
print("Наилучшее значение f1_score с новым признаком:", round(best_f1_score, 2))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import xgboost as xgb

# Создание и обучение RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Предсказание и оценка на тестовых данных
y_test = test_data['price_range']
y_pred_rf = rf_model.predict(X_test)
rf_f1_weighted = f1_score(y_test, y_pred_rf, average='weighted')
print("f1_weighted RandomForestClassifier:", round(rf_f1_weighted, 2))

# Создание и обучение XGBoost
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

# Предсказание и оценка на тестовых данных
y_pred_xgb = xgb_model.predict(X_test)
xgb_f1_weighted = f1_score(y_test, y_pred_xgb, average='weighted')
print("f1_weighted XGBoost:", round(xgb_f1_weighted, 2))

# Сохранение лучшей модели
best_model = rf_model if rf_f1_weighted > xgb_f1_weighted else xgb_model
best_f1_weighted = max(rf_f1_weighted, xgb_f1_weighted)
print("Наилучшая f1_weighted модели:", round(best_f1_weighted, 2))

a) (0.3 баллa). Далее используйте только категориальные столбцы. Закодируйте их с помощью One-hot encoding с учетом того, что мы не хотим получить мультиколлинеарности в новых данных. Сколько получилось числовых столбцов из исходных категориальных? Кодируйте и df_train, и df_test.

б) (0.4 балла). Попытаемся по характеристикам кошки (бывшие категориальные, а теперь - числовые столбцы) предсказать, прошла она полосу препятствий или нет.


Сформируйте из df_train матрицу объект-признак X и вектор ответов y.


Обучите решающее дерево (DecisionTreeClassifier из библиотеки sklearn.tree) глубины 5 с энтропийным критерием информативности на закодированных в пункте а) тренировочных данных по кросс-валидации с тремя фолдами, метрика качества - roc-auc.


Чему равен roc-auc, усредненный по фолдам? Ответ округлите до десятых.


Комментарий: остальные гиперпараметры дерева оставьте дефолтными (splitter='best', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, ccp_alpha=0.0)

Задание 6 (1.5 балла максимум).


a) (0.25 балла). Подберите глубину решающего дерева (max_depth), перебирая глубину от 2 до 20 с шагом 1 и используя перебор по сетке (GridSearchCV из библиотеки sklearn.model_selection) с тремя фолдами и метрикой качества - roc-auc. В ответ запишите наилучшее среди искомых значение max_depth.


Комментарий: остальные гиперпараметры дерева оставьте дефолтными (splitter='best', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, ccp_alpha=0.0)

б) (0.5 балла). Добавьте к данным новый признак cat_bio, содержащий в качестве значений пары значений из столбца type и столбца group. Например, если кошка имеет type='wild' и  group='group B', то в cat_bio будет стоять строка '(wild, group B)'. Примените OneHotEncoding (с учетом того, что мы не хотим получить мультиколлинеарности в новых данных) к столбцам 'cat_bio', 'education', 'meal', 'preparation course', а затем обучите решающее дерево глубины 5 с энтропийным критерием информативности на полученных после кодирования данных. Чему равен roc-auc? Ответ округлите до сотых.

Комментарий: остальные гиперпараметры дерева оставьте дефолтными (splitter='best', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, ccp_alpha=0.0)

в) (0.75 балла). Теперь вы можете использовать любую модель машинного обучения для решения задачи. Также можете делать любую другую обработку признаков. Ваша задача - получить наилучшее качество (ROC_AUC).

Качество проверяется на тестовых данных.

ROC_AUC > 0.7 - 0.25 балла
ROC_AUC > 0.74 - 0.75 балла

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Загрузка данных
df_train = pd.read_csv('TrainData.csv')
df_test = pd.read_csv('TestData.csv')

# Определение категориальных столбцов
categorical_columns = df_train.select_dtypes(include=['object']).columns

# One-Hot Encoding с параметром drop='first'
encoder = OneHotEncoder(drop='first', sparse=False)
encoded_train = encoder.fit_transform(df_train[categorical_columns])
encoded_test = encoder.transform(df_test[categorical_columns])

# Преобразование закодированных данных обратно в DataFrame
encoded_train_df = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out(categorical_columns))
encoded_test_df = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out(categorical_columns))

# Объединение закодированных данных с оригинальными DataFrame, исключая исходные категориальные столбцы
df_train_encoded = pd.concat([df_train.drop(columns=categorical_columns), encoded_train_df], axis=1)
df_test_encoded = pd.concat([df_test.drop(columns=categorical_columns), encoded_test_df], axis=1)

# Количество новых столбцов, полученных в результате кодирования
num_new_columns = len(encoded_train_df.columns)

print(f"Количество новых столбцов из исходных категориальных: {num_new_columns}")

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, make_scorer

# Формирование матрицы объект-признак X и вектора ответов y
X = df_train_encoded.drop(columns=['target'])
y = df_train_encoded['target']

# Инициализация модели решающего дерева
tree = DecisionTreeClassifier(max_depth=5, criterion='entropy', random_state=42)

# Оценка модели с использованием кросс-валидации
roc_auc_scorer = make_scorer(roc_auc_score)
scores = cross_val_score(tree, X, y, cv=3, scoring=roc_auc_scorer)

# Усредненное значение ROC-AUC
average_roc_auc = scores.mean()
print(f"Усредненное значение ROC-AUC по фолдам: {average_roc_auc:.1f}")

In [ ]:
from sklearn.model_selection import GridSearchCV

# Настройка сетки гиперпараметров
param_grid = {'max_depth': range(2, 21)}

# Инициализация модели решающего дерева
tree = DecisionTreeClassifier(criterion='entropy', random_state=42)

# Инициализация GridSearchCV
grid_search = GridSearchCV(tree, param_grid, cv=3, scoring=roc_auc_scorer)
grid_search.fit(X, y)

# Лучшее значение max_depth
best_max_depth = grid_search.best_params_['max_depth']
print(f"Наилучшее значение max_depth: {best_max_depth}")


In [ ]:
# Добавление нового признака cat_bio
df_train['cat_bio'] = list(zip(df_train['type'], df_train['group']))
df_test['cat_bio'] = list(zip(df_test['type'], df_test['group']))

# Применение OneHotEncoding к новым признакам и другим категориальным столбцам
new_categorical_columns = ['cat_bio', 'education', 'meal', 'preparation course']
encoder = OneHotEncoder(drop='first', sparse=False)
encoded_train_new = encoder.fit_transform(df_train[new_categorical_columns])
encoded_test_new = encoder.transform(df_test[new_categorical_columns])

# Преобразование закодированных данных обратно в DataFrame
encoded_train_new_df = pd.DataFrame(encoded_train_new, columns=encoder.get_feature_names_out(new_categorical_columns))
encoded_test_new_df = pd.DataFrame(encoded_test_new, columns=encoder.get_feature_names_out(new_categorical_columns))

# Объединение закодированных данных с оригинальными DataFrame
df_train_new_encoded = pd.concat([df_train.drop(columns=new_categorical_columns), encoded_train_new_df], axis=1)
df_test_new_encoded = pd.concat([df_test.drop(columns=new_categorical_columns), encoded_test_new_df], axis=1)

# Формирование матрицы объект-признак X_new и вектора ответов y
X_new = df_train_new_encoded.drop(columns=['target'])
y = df_train_new_encoded['target']

# Обучение модели и оценка качества
tree_new = DecisionTreeClassifier(max_depth=5, criterion='entropy', random_state=42)
scores_new = cross_val_score(tree_new, X_new, y, cv=3, scoring=roc_auc_scorer)
average_roc_auc_new = scores_new.mean()
print(f"Усредненное значение ROC-AUC по фолдам с новым признаком: {average_roc_auc_new:.2f}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# Использование случайного леса
model = RandomForestClassifier(random_state=42)
model.fit(X_new, y)

# Предсказания на тестовых данных
X_test_new = df_test_new_encoded.drop(columns=['target'])
y_test = df_test_new_encoded['target']
y_pred = model.predict_proba(X_test_new)[:, 1]

# Оценка качества на тестовых данных
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC_AUC на тестовых данных: {roc_auc:.2f}")
